In [1]:
import pandas as pd
import sys 
import numpy as np

sys.path.append('/home/alexabades/recsys')
from src.data.Contextual import PreProcessDataNCFContextual
datapath = "../data/raw/frappe/"
df_frappe = PreProcessDataNCFContextual(datapath)
# df_frappe.save_data("frappeCtxA")

In [12]:
df_frappe.columns

['user',
 'item',
 'rating',
 'daytime',
 'weather',
 'isweekend',
 'homework',
 'cnt']

In [16]:
cols = ['cnt', 'daytime_afternoon', 'daytime_evening',
       'daytime_morning', 'daytime_night', 'daytime_noon', 'daytime_sunrise',
       'daytime_sunset', 'weather_cloudy', 'weather_drizzle', 'weather_foggy',
       'weather_rainy', 'weather_snowy', 'weather_stormy', 'weather_sunny',
       'weather_unknown', 'isweekend_weekend', 'isweekend_workday',
       'homework_home', 'homework_unknown', 'homework_work']
print(cols)

['cnt', 'daytime_afternoon', 'daytime_evening', 'daytime_morning', 'daytime_night', 'daytime_noon', 'daytime_sunrise', 'daytime_sunset', 'weather_cloudy', 'weather_drizzle', 'weather_foggy', 'weather_rainy', 'weather_snowy', 'weather_stormy', 'weather_sunny', 'weather_unknown', 'isweekend_weekend', 'isweekend_workday', 'homework_home', 'homework_unknown', 'homework_work']


In [5]:
len(df_frappe.data.user.unique()), df_frappe.data.user.max()

(651, 650)

In [8]:
len(df_frappe.data.item.unique()), df_frappe.data.item.max()

(1127, 1126)

In [10]:
len(df_frappe.train_ratings.item.unique()), df_frappe.train_ratings.item.unique().max()

(1127, 1126)

In [11]:
df_frappe.save_data("frappeCtxA")

/home/alexabades/recsys/src/data/processed/frappeCtxA/frappeCtxA


In [3]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Example DataFrame
df = pd.DataFrame({
    'column1': [1, 2, 3, 4, 5],
    'column2': [10, 20, 30, 40, 50],
    'column3': [100, 200, 300, 400, 500]
})

# Columns to normalize
columns = ['column1', 'column2']

# Initializing the MinMaxScaler
scaler = MinMaxScaler()

# Applying the scaler to the selected columns
df[columns] = scaler.fit_transform(df[columns])

print(df)


   column1  column2  column3
0     0.00     0.00      100
1     0.25     0.25      200
2     0.50     0.50      300
3     0.75     0.75      400
4     1.00     1.00      500


I have some troubles achieveing a train test split for my dataset in recomendation sistems. 
If I am dealing with sparse data, where I have some items and users with low interaction. Do we have to ensure in the train test split that at least 1 item/user appears at least once in the train and test set? 
This problem may occur also with datasets with more interactions, but in order to ilustarte in a simple example, I'll create a small dataset: 

Example:

df = pd.DataFrame({
    'userID': [0, 0, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5],
    'itemID': [0, 1, 0, 2, 3, 4, 1, 5, 2, 4, 3, 5],
})

If we check the num of iteraction by user and the times an item has been interacted: 

num_interactions_by_user = df.groupby('userID').count()
num_interaction_by_item = df.groupby('itemID').count()

num of iteraction by user
userID     count       
0            2
1            2
2            2
3            2
4            2
5            2

num times an item has been interacted
itemID     count   
0            2 <br>
1            2
2            2
3            2
4            2
5            2

We can see that some of our samples have only 2. If we wish to train test split kind of randomly we can do something like:
for i in df.userID.unique():
   test_idx = df.userID.random(1)

The problem of this approach is that we may end up with some items only in the test set, which is not obtimal. For example 

Being the index of the dataframe idx = list(df.index) we could en up with:
idx_test = [0, 2, 4, 6, 8, 10]
idx_train = [1, 3, 5, 7, 9, 11]

We can see that in this case the test set will end up like:

    userID  itemID
0        0       0
2        1       0
4        2       3
6        3       1
8        4       2
10       5       3

And the train set like:

    userID  itemID
1        0       1
3        1       2
5        2       4
7        3       5
9        4       4
11       5       5

And wheras we might have been able to add at least one user in each set we fail to add at least one item both sets.

df[np.isin(df.index, idx_test)]['itemID'].unique() = [0 3 1 2]
df[np.isin(df.index, idx_train)]['itemID'].unique() = [1 2 4 5]

We fail to add itemIDs 0 and 3 in the train set and we fail to add the itemIDs 4 and 5 in the test set. I believe that missing some items in the test set won't supose that much trouble. But if we miss one items in the train set we could face the cold start problem for that specific Item, as the model would not have seen this item in the training process.

In [2]:
import pandas as pd
import numpy as np

# Define the DataFrame
df = pd.DataFrame({
    'userID': [0, 0, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5],
    'itemID': [0, 1, 0, 2, 3, 4, 1, 5, 2, 4, 3, 5],
})

# Create empty lists to hold the indices for the train and test sets
idx_train = []
idx_test = []

# Loop through each user to ensure they appear in both sets
for user_id in df['userID'].unique():
    user_interactions = df[df['userID'] == user_id]
    
    # Ensure at least one interaction per user goes into each set
    train_interaction = user_interactions.sample(n=1, random_state=42)  # Sample one interaction for training
    test_interaction = user_interactions.drop(train_interaction.index)  # Use the other for testing
    
    # Append indices to the appropriate lists
    idx_train.extend(train_interaction.index.tolist())
    idx_test.extend(test_interaction.index.tolist())

# Verify that each item appears in both sets
train_items = df.loc[idx_train, 'itemID'].unique()
test_items = df.loc[idx_test, 'itemID'].unique()

set(train_items), set(test_items), set(train_items) | set(test_items), set(train_items) & set(test_items)


({1, 2, 4, 5}, {0, 1, 2, 3}, {0, 1, 2, 3, 4, 5}, {1, 2})

In [3]:
def get_element_idx_given_min_interactions(data, filter_column, ratings_column, min_interactions):
    data_groupedby_item_interaction = data.groupby(filter_column)[
        [ratings_column]
    ].count()
    iDs_low_interaction = data_groupedby_item_interaction[
        data_groupedby_item_interaction[ratings_column] <= min_interactions
    ].index

    idx_few_interactions = np.isin(data[filter_column], iDs_low_interaction)
    return idx_few_interactions

In [4]:
def iterative_cleaning(data, user_column, item_column, ratings_column, min_interactions):
  clean = False 
  cnt = 0 
  data_iter = data.copy()
  while not clean:
    item_indx = get_element_idx_given_min_interactions(data_iter, item_column, ratings_column, min_interactions)
    data_clean_items = data_iter[~item_indx]
    user_indx = get_element_idx_given_min_interactions(data_clean_items, user_column, ratings_column, min_interactions)
    data_clean_items_users = data_clean_items[~user_indx]
    data_iter = data_clean_items_users.copy()
    numMinInteractionsItems = data_clean_items_users.groupby(item_column)[ratings_column].count().min()
    numMinInteractionsUsres = data_clean_items_users.groupby(user_column)[ratings_column].count().min()
    # clean = True
    if numMinInteractionsItems > min_interactions and numMinInteractionsUsres > min_interactions:
      clean = True
    if cnt == 10:
      print(f"Iteration {cnt}")
    cnt += 1
  return data_clean_items_users

## Non continuous sequence
User & Item IDs are non continuous sequence because some users or items have been removed during data cleaning. <br>
ID Mapping: To deal with the non-continuous nature of user IDs, create a mapping between the original user IDs and a new set of indices that are continuous. For instance, you could map your 951 user IDs to indices ranging from 0 to 950. This mapping would be used when feeding data into the model and when interpreting the model's output.

In [34]:
train_data = df_frappe.train_ratings.copy()
test_data = df_frappe.test_ratings.copy()
unique_user_id = train_data['user'].unique()
unique_item_id = train_data['item'].unique()
is_user_id_sequence_continuous = (max(unique_user_id) - min(unique_user_id) + 1) == len(unique_user_id)
is_item_id_sequence_continuous = (max(train_data['item'].unique()) - min(train_data['item'].unique()) + 1) == len(train_data['item'].unique())
print(is_user_id_sequence_continuous)

def _map_elemetIDs(data, column_to_map:str):
    unique_element_ids = sorted(data[column_to_map].unique())
    element_id_to_index = {element_id: index for index, element_id in enumerate(unique_element_ids)}
    return element_id_to_index

user_id_to_index = _map_elemetIDs(train_data, 'user')

# len(item_id_to_index.keys()), max(item_id_to_index.keys())
# train_data['user'] = train_data['user'].apply(lambda x: item_id_to_index[x])
# is_user_id_sequence_continuous = (max(train_data['user'].unique()) - min(train_data['user'].unique()) + 1) == len(train_data['user'].unique())
# print(is_user_id_sequence_continuous)


False


In [35]:
train_data['user'].map(user_id_to_index)

0          0
1          1
2          2
3          3
4          4
        ... 
96198    110
96199     37
96200    181
96201    449
96202    362
Name: user, Length: 89099, dtype: int64

In [28]:
len(train_data['user'].unique())

869

In [26]:
isItemIDSequenceContinuous = max(train_data['user'].unique()) + 1 == len(train_data['user'].unique())
isItemIDSequenceContinuous

True

In [21]:
len(train_data['user'].unique()), train_data['user'].unique().max()

(869, 868)

In [ ]:

index_to_item_id = {index: item_id for item_id, index in item_id_to_index.items()}


unique_user_ids = sorted(train_data['item'].unique())
user_id_to_index = {user_id: index for index, user_id in enumerate(unique_user_ids)}
index_to_user_id = {index: user_id for user_id, index in user_id_to_index.items()}

In [4]:
item_id_to_index

{0: 0,
 1: 1,
 2: 2,
 3: 3,
 4: 4,
 5: 5,
 6: 6,
 7: 7,
 8: 8,
 9: 9,
 10: 10,
 11: 11,
 12: 12,
 13: 13,
 14: 14,
 15: 15,
 16: 16,
 17: 17,
 18: 18,
 19: 19,
 20: 20,
 21: 21,
 22: 22,
 23: 23,
 24: 24,
 25: 25,
 26: 26,
 27: 27,
 28: 28,
 29: 29,
 30: 30,
 31: 31,
 32: 32,
 33: 33,
 34: 34,
 35: 35,
 36: 36,
 37: 37,
 38: 38,
 39: 39,
 40: 40,
 41: 41,
 42: 42,
 43: 43,
 44: 44,
 45: 45,
 46: 46,
 47: 47,
 48: 48,
 49: 49,
 50: 50,
 51: 51,
 52: 52,
 53: 53,
 54: 54,
 55: 55,
 56: 56,
 57: 57,
 58: 58,
 59: 59,
 60: 60,
 61: 61,
 62: 62,
 63: 63,
 64: 64,
 65: 65,
 66: 66,
 67: 67,
 68: 68,
 69: 69,
 70: 70,
 71: 71,
 72: 72,
 73: 73,
 74: 74,
 75: 75,
 76: 76,
 77: 77,
 78: 78,
 79: 79,
 80: 80,
 81: 81,
 82: 82,
 83: 83,
 84: 84,
 85: 85,
 86: 86,
 87: 87,
 88: 88,
 89: 89,
 90: 90,
 91: 91,
 92: 92,
 93: 93,
 94: 94,
 95: 95,
 96: 96,
 97: 97,
 98: 98,
 99: 99,
 100: 100,
 101: 101,
 102: 102,
 103: 103,
 104: 104,
 105: 105,
 106: 106,
 107: 107,
 108: 108,
 109: 109,
 110: 110,

## Ensure K items in the train set

In [ ]:
df2 = pd.DataFrame({
    'userID': [0, 0, 0, 0, 0, 2, 2, 2, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5],
    'itemID': [0, 1, 5, 7, 6, 0, 1, 7, 0, 2, 7, 5, 1, 6, 0, 1, 7, 5, 0, 6, 0, 1, 5, 6, 8],
    'rating': [4.1, 1, 5, 4.2, 4.1, 3.8, 1, 3.7, 1.3, 2, 2.7, 5, 1, 2.6, 2, 1.7, 3.7, 5, 2, 4.6, 2.0, 1, 5, 3.6, 3.8],
})
user_column = 'userID'
item_column = 'itemID'
ratings_column = 'rating'
min_interactions = 4

In [ ]:
df2.groupby('userID')['itemID'].count()
df2.groupby('itemID')['userID'].count()

In [11]:
df = pd.DataFrame({
    'userID': [0, 0, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5],
    'itemID': [0, 1, 0, 2, 3, 4, 1, 5, 2, 4, 3, 5],
})

In [68]:
import random
unique_users_id = df.userID.unique()
test_idx = []
train_idx = []
for user in unique_users_id:
  item_found = False
  items_imteracted_by_user = list(df[df['userID'] == user]['itemID'].values)
  while not item_found:
    selected_item = random.choice(items_imteracted_by_user)  
    items_imteracted_by_user.remove(selected_item)    
    test_sample = df[(df['userID'] == user) & (df['itemID'] == selected_item)]
    train_samples = df[(df['userID'] == user) & (df['itemID'] != selected_item)]
    test_sample_idx = test_sample.index[0]
    train_samples_idx = train_samples.index[0]
    
    if test_sample_idx not in test_idx:
      test_idx.append(test_sample_idx)
      item_found = True
    if not items_imteracted_by_user:
      print('No items remaining')
      break

In [69]:
df_test = df[np.isin(df.index, test_idx)]
df_train = df[~np.isin(df.index, test_idx)]

As we are iterating over the users we are ensuring that the user apears both in the train and the test set.

In [81]:
np.all(np.isin(df_train['userID'].unique(), df['userID'].unique()))

True

The problem comes when we check the items. Ideally the itemIds from train and test would be the same, which would mean that all items appear once in the train and test set. 
Nevertheless, if we are in a situation where an item cannot go to both, we should include it in the train. Therefore, ideally we should obtain something like:

df_train.group

After this, we can check 

In [74]:
df_train.groupby('userID').count()

,itemID
userID,
0,1
1,1
2,1
3,1
4,1
5,1


In [79]:
np.isin([2, 3, 1], [1, 2, 4]) # if every element from the first array is in the second array 

array([ True, False,  True])

In [82]:
unique_itemID_train = df_train['itemID'].unique()
unique_itemID_data = df['itemID'].unique()
np.all(np.isin(unique_itemID_data, unique_itemID_train))

False

In [96]:
items_missing_train = set(unique_itemID_data) - set(unique_itemID_train)
items_missing_train

{0, 4}

In [87]:
K = 2
df_train.groupby('itemID').count() >= K 
df_train.groupby('userID').count() >= K 

,userID
itemID,
1,True
2,True
3,True
5,True


In [ ]:

np.all(np.isin(df_train['itemID'].unique(), df_test['itemID'].unique()))

In [66]:
df_train['itemID'].unique(), df_test['itemID'].unique()

(array([1, 0, 4, 5, 3]), array([0, 2, 3, 1, 5]))

In [25]:
a = [1, 2, 3, 4]
b = set(a)

In [29]:
import random

a = [1, 2, 3, 4, 5, 6, 7]
selected_item = random.choice(a)  #
a.remove(selected_item)    

In [55]:
cnt = 0
found = False
while not found:
  cnt += 1
  if cnt == 4:
    print('count break')
    break

count break


In [97]:
import pandas as pd
import numpy as np

# Define the DataFrame again after reset
df = pd.DataFrame({
    'userID': [0, 0, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5],
    'itemID': [0, 1, 0, 2, 3, 4, 1, 5, 2, 4, 3, 5],
})

# Setting K value
K = 2

# Step 1: Initial Filtering - Remove users and items with fewer than K+1 interactions
# Count interactions by user and item
user_counts = df.groupby('userID')['itemID'].count()
item_counts = df.groupby('itemID')['userID'].count()

# Filter users and items with at least K+1 interactions
valid_users = user_counts[user_counts >= K+1].index
valid_items = item_counts[item_counts >= K+1].index

# Filter the DataFrame
filtered_df = df[df['userID'].isin(valid_users) & df['itemID'].isin(valid_items)]

# Step 2: Assuming iterative cleaning has been done as part of the initial filtering for simplicity

# Step 3: Allocating interactions to training and test sets
# Initialize train and test indices
train_idx = []
test_idx = []

for user in filtered_df['userID'].unique():
    user_items = filtered_df[filtered_df['userID'] == user]['itemID'].unique()
    
    # Randomly select an item for the test set that has not been selected before
    test_item_selected = False
    while not test_item_selected and user_items.size > 0:
        test_item = np.random.choice(user_items)
        test_item_instances = filtered_df[(filtered_df['userID'] == user) & (filtered_df['itemID'] == test_item)]
        
        # Ensure this item has enough interactions to be split between train and test
        if len(test_item_instances) > 1 or (test_item in valid_items and item_counts[test_item] > K):
            test_idx.append(test_item_instances.index[0])  # Take one instance for the test set
            test_item_selected = True
        
        # Update user_items to prevent re-selection of the test item
        user_items = user_items[user_items != test_item]

    # Remaining interactions go to the training set
    remaining_indices = filtered_df[(filtered_df['userID'] == user) & (~filtered_df['itemID'].isin([test_item]))].index
    train_idx.extend(remaining_indices.tolist())

# Convert indices to DataFrame subsets
df_train = df.loc[train_idx]
df_test = df.loc[test_idx]

# Verification
unique_itemID_train = df_train['itemID'].unique()
unique_itemID_test = df_test['itemID'].unique()
unique_userID_train = df_train['userID'].unique()
unique_userID_test = df_test['userID'].unique()

# Check if all items and users from original filtered dataset appear in both sets
all_items_in_both_sets = np.all(np.isin(unique_itemID_train, unique_itemID_test)) and np.all(np.isin(unique_itemID_test, unique_itemID_train))
all_users_in_both_sets = np.all(np.isin(unique_userID_train, unique_userID_test)) and np.all(np.isin(unique_userID_test, unique_userID_train))

# Check if the train set satisfies the K interactions constraint
train_item_counts_satisfy_K = np.all(df_train.groupby('itemID')['userID'].count() >= K)
train_user_counts_satisfy_K = np.all(df_train.groupby('userID')['itemID'].count() >= K)

all_items_in_both_sets, all_users_in_both_sets, train_item_counts_satisfy_K, train_user_counts_satisfy_K


(True, True, True, True)